In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch 
import os
import glob
from matplotlib import pyplot as plt
import os
import torch
import collections
# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from datasets import example_datasets, to_numpy
from torch_models import  MixtureOfTruncNormModel, torch_bpr_uncurried, deterministic_bpr, SpatialWaves
from torch_perturb.torch_pert_topk import PerturbedTopK
from distributions import QuantizedNormal



In [ ]:
base_dir = '/cluster/tufts/hugheslab/kheuto01/large_synth_4000'
exps = glob.glob(base_dir + '/*')
len(exps)

In [ ]:
def make_report(model, exp, data=None, title=None):
    model.load_state_dict(torch.load(exp + '/model.pth'))
    losses = torch.load(exp + '/losses.pth')
    bprs = torch.load(exp + '/bprs.pth')
    nlls = torch.load(exp + '/nlls.pth')

    losses = np.array([loss.detach().numpy() for loss in losses])
    bprs = np.array([bpr.detach().numpy() for bpr in bprs])
    nlls = np.array([nll.detach().numpy() for nll in nlls])

    
    # Make an html page with plots of loss, bpr, nll
    fig, ax = plt.subplots(2, 2, figsize=(10, 10))
    ax[0, 0].plot(losses)
    ax[0, 0].set_title('Loss')
    ax[0, 1].plot(bprs)
    ax[0, 1].set_title('BPR')
    ax[1, 0].plot(nlls)
    ax[1, 0].set_title('NLL')
    if data is not None:
        model.plot_learned(data=data, ax=ax[1, 1])
    if title is not None:
        fig.suptitle(title)
    fig.savefig(exp + '/report.png')
    return model
    

In [ ]:
max_bpr = 0
for e,exp in enumerate(exps):



    if e % 10 == 0:
        print(f'Processing {e} / {len(exps)}')

    # extract bw number and new number from model name
    bw = exp.split('bw')[1].split('_')[0]
    nw = exp.split('nw')[1].split('_')[0]
    title = f'BPR Weight: {bw}, NLL Weight: {nw}'
    model = SpatialWaves(num_waves=1, low=0, high=100000)
    model = make_report(model, exp, None, title=title)
    losses = torch.load(exp + '/losses.pth')
    bprs = torch.load(exp + '/bprs.pth')
    nlls = torch.load(exp + '/nlls.pth')
    bpr = bprs[-1]
    if bpr>=max_bpr:
        max_bpr = bpr
        best_exp = exp
        

In [ ]:
plt.imshow(lat_S.reshape(data_shape))

In [ ]:
time_T = torch.tensor(time_T, dtype=torch.float32)
lat_S = torch.tensor(lat_S, dtype=torch.float32)
lon_S = torch.tensor(lon_S, dtype=torch.float32)
pop_S = torch.tensor(pop_S, dtype=torch.float32)
S = pop_S.shape[0]
T = time_T.shape[0]
W = 2

In [ ]:
times_by_score = {'50':{'50':[], '100':[], '200':[]}, '100':{'50':[], '100':[], '200':[]}, '200':{'50':[], '100':[], '200':[]}}
for e,exp in enumerate(exps):
    losses = torch.load(exp + '/losses.pth')
    bprs = torch.load(exp + '/bprs.pth')
    nlls = torch.load(exp + '/nlls.pth')
    times = torch.load(exp + '/times.pth')

    losses = np.array([loss.detach().numpy() for loss in losses])
    bprs = np.array([bpr.detach().numpy() for bpr in bprs])
    nlls = np.array([nll.detach().numpy() for nll in nlls])
    try:
        nss = exp.split('nss')[1].split('_')[0]
        nps = exp.split('nps')[1].split('_')[0]

        mean_time = np.mean(times)
        num= len(times)
        if num>500:
            times_by_score[nss][nps].append(mean_time)
            print(f'Nss: {nss}, Nps: {nps}, Mean Time: {mean_time}, num:{len(times)} exp: {exp}')
    except:
        continue